In [ ]:
pip install langgraph langchain langchain-core langchain-google-genai langchain-community python-dotenv

In [2]:
# GOOGLE_API_KEY="__"

In [ ]:
############Memory in Langgraph############

In [3]:
#  Bot with Persistence/Memory (Module 3)
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver

load_dotenv()

memory = MemorySaver() # for a demo, prod : any db!

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key="EnteryourownKey")

In [4]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: BasicChatState):
    return {
       "messages": [llm.invoke(state["messages"])]
    }

graph = StateGraph(BasicChatState)
graph.add_node("chatbot", chatbot)
graph.add_edge("chatbot", END)
graph.set_entry_point("chatbot")

app = graph.compile(checkpointer=memory)

In [6]:
config = {"configurable": {
    "thread_id": 1
}}
while True:
    user_input = input("User: ")
    if user_input in ["exit", "end"]:
        break
    else:
        result = app.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config=config)

        print("AI: " + result["messages"][-1].content)

User:  hi my name is devender i bought pen yesterday


AI: Hi Devender! Nice to meet you.

That's great you bought a pen yesterday! What kind did you get, or is there anything specific you wanted to tell me about it?


User:  oh i did not bought pen


AI: My apologies, Devender! Thanks for clarifying. I misunderstood.

So you *didn't* buy a pen yesterday. Is there something else you wanted to share or talk about?


User:  sorry what is my name


AI: Your name is **Devender**.


User:  exit


In [7]:
############ HITL(Human in the loop)in Langgraph############

In [8]:
from langgraph.graph import StateGraph, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

In [9]:
memory = MemorySaver()
class CampaignState(TypedDict):
    customer_name: str
    email_draft: str
    subject: str
    campaign_type: str
    approved: bool

In [10]:
def analyze_customer(state: CampaignState):
    """AI analyzes customer data and decides campaign type"""
    print(f"\n🔍 Analyzing customer: {state['customer_name']}")

    # Simulate AI analysis
    campaign_type = "promotional"  # Could be "follow-up", "reminder", etc.

    return Command(
        goto="draft_email",
        update={"campaign_type": campaign_type}
    )

In [11]:
def draft_email(state: CampaignState):
    """AI drafts an email based on campaign type"""
    print(f"✍️  Drafting {state['campaign_type']} email...")

    # AI generates email content
    drafts = {
        "promotional": {
            "subject": f"Special Offer for {state['customer_name']}!",
            "body": f"Hi {state['customer_name']},\n\nWe have an exclusive 30% discount just for you! Valid until Friday.\n\nBest regards,\nSales Team"
        },
        "reminder": {
            "subject": f"Don't forget - Your cart is waiting!",
            "body": f"Hi {state['customer_name']},\n\nYou left items in your cart. Complete your purchase now!\n\nThanks,\nSupport Team"
        }
    }

    draft = drafts.get(state['campaign_type'], drafts['promotional'])

    return Command(
        goto="human_review",
        update={
            "subject": draft["subject"],
            "email_draft": draft["body"]
        }
    )

In [12]:
def human_review(state: CampaignState):
    """Human reviews and approves/rejects the email"""
    print("\n" + "="*60)
    print("📧 EMAIL DRAFT FOR REVIEW")
    print("="*60)
    print(f"To: {state['customer_name']}")
    print(f"Subject: {state['subject']}")
    print(f"\n{state['email_draft']}")
    print("="*60)

    # Interrupt for human decision
    decision = interrupt("Approve this email? Type 'send' or 'reject'")

    print(f"\n✅ Human decision: {decision}")

    if decision.lower() == "send":
        return Command(
            goto="send_email",
            update={"approved": True}
        )
    else:
        return Command(
            goto="log_rejection",
            update={"approved": False}
        )

In [13]:
def send_email(state: CampaignState):
    """Send the approved email"""
    print(f"\n📤 Sending email to {state['customer_name']}...")
    print("✅ Email sent successfully!")
    return Command(goto=END)

def log_rejection(state: CampaignState):
    """Log that email was rejected"""
    print(f"\n❌ Email rejected for {state['customer_name']}")
    print("📝 Logged for review by marketing team")
    return Command(goto=END)

In [14]:
graph = StateGraph(CampaignState)
graph.add_node("analyze_customer",analyze_customer)
graph.add_node("draft_email",draft_email)
graph.add_node("human_review",human_review)
graph.add_node("send_email",send_email)
graph.add_node("log_rejection",log_rejection)

graph.set_entry_point("analyze_customer")

app = graph.compile(checkpointer=memory)

# Demo execution
config = {"configurable": {"thread_id": "campaign_001"}}

In [15]:
# Step 1: Run until human review needed
initial_state = {
    "customer_name": "Alice Johnson",
    "email_draft": "",
    "subject": "",
    "campaign_type": "",
    "approved": False
}

result = app.invoke(initial_state, config)

# print("\n⏸️  Workflow paused for human review...")

# # Step 2: Human makes decision
decision = input("\n👤 Your decision (send/reject): ").strip().lower()

# # Step 3: Resume workflow
print(f"\n▶️  Resuming workflow with decision: {decision}")
final_result = app.invoke(Command(resume=decision), config)

print("\n✨ Workflow completed!")
print(f"Final status: {'Sent' if final_result.get('approved') else 'Rejected'}")



🔍 Analyzing customer: Alice Johnson
✍️  Drafting promotional email...

📧 EMAIL DRAFT FOR REVIEW
To: Alice Johnson
Subject: Special Offer for Alice Johnson!

Hi Alice Johnson,

We have an exclusive 30% discount just for you! Valid until Friday.

Best regards,
Sales Team



👤 Your decision (send/reject):  send



▶️  Resuming workflow with decision: send

📧 EMAIL DRAFT FOR REVIEW
To: Alice Johnson
Subject: Special Offer for Alice Johnson!

Hi Alice Johnson,

We have an exclusive 30% discount just for you! Valid until Friday.

Best regards,
Sales Team

✅ Human decision: send

📤 Sending email to Alice Johnson...
✅ Email sent successfully!

✨ Workflow completed!
Final status: Sent
